# Data Image or Augmentation

In [1]:
import os
os.chdir("C:/Users/Sneha/Desktop/InternProject/brain_tumor_dataset")

In [2]:
os.getcwd()

'C:\\Users\\Sneha\\Desktop\\InternProject\\brain_tumor_dataset'

# Importing ImageDataGenerator Library

In [3]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

Using TensorFlow backend.
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
E:\Anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

# Applying ImageDataGenerator to train and testsets

In [4]:
x_train = train_datagen.flow_from_directory(r'C:/Users/Sneha/Desktop/InternProject/brain_tumor_dataset/train_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                     class_mode = 'binary')
x_test = test_datagen.flow_from_directory(r'C:/Users/Sneha/Desktop/InternProject/brain_tumor_dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 180 images belonging to 2 classes.
Found 73 images belonging to 2 classes.


In [5]:
print(x_train.class_indices)

{'no': 0, 'yes': 1}


In [6]:
print(x_test.class_indices)

{'no': 0, 'yes': 1}
